In [1]:
import tensorflow as tf 
print(tf.__version__)

2.1.0


In [2]:
import tensorflow as tf

# Check that GPU is available: cf. https://colab.research.google.com/notebooks/gpu.ipynb
assert(tf.test.is_gpu_available())

tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(False) # Start with XLA disabled.

def load_data():
  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
  x_train = x_train.astype('float32') / 256
  x_test = x_test.astype('float32') / 256

  # Convert class vectors to binary class matrices.
  y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
  y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)
  return ((x_train, y_train), (x_test, y_test))

(x_train, y_train), (x_test, y_test) = load_data()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [3]:
print(x_train)

[[[[0.23046875 0.2421875  0.24609375]
   [0.16796875 0.1796875  0.17578125]
   [0.1953125  0.1875     0.16796875]
   ...
   [0.6171875  0.515625   0.421875  ]
   [0.59375    0.48828125 0.3984375 ]
   [0.578125   0.484375   0.40234375]]

  [[0.0625     0.078125   0.078125  ]
   [0.         0.         0.        ]
   [0.0703125  0.03125    0.        ]
   ...
   [0.48046875 0.34375    0.21484375]
   [0.46484375 0.32421875 0.1953125 ]
   [0.4765625  0.33984375 0.22265625]]

  [[0.09765625 0.09375    0.08203125]
   [0.0625     0.02734375 0.        ]
   [0.19140625 0.10546875 0.03125   ]
   ...
   [0.4609375  0.328125   0.1953125 ]
   [0.46875    0.328125   0.1953125 ]
   [0.42578125 0.28515625 0.1640625 ]]

  ...

  [[0.8125     0.6640625  0.375     ]
   [0.78515625 0.59765625 0.1328125 ]
   [0.7734375  0.62890625 0.1015625 ]
   ...
   [0.625      0.51953125 0.2734375 ]
   [0.21875    0.12109375 0.02734375]
   [0.20703125 0.1328125  0.078125  ]]

  [[0.703125   0.54296875 0.375     ]
   [0.6

In [8]:
import tensorflow as tf
print(tf.__version__)
from keras.datasets import cifar10
import matplotlib

2.1.0


In [9]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [10]:
def generate_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(32, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(64, (3, 3), padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Activation('softmax')
  ])

model = generate_model()

In [11]:
def compile_model(model):
  opt = tf.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
  model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
  return model

model = compile_model(model)

def train_model(model, x_train, y_train, x_test, y_test, epochs=25):
  model.fit(x_train, y_train, batch_size=256, epochs=epochs, validation_data=(x_test, y_test), shuffle=True)

def warmup(model, x_train, y_train, x_test, y_test):
  # Warm up the JIT, we do not wish to measure the compilation time.
  initial_weights = model.get_weights()
  train_model(model, x_train, y_train, x_test, y_test, epochs=1)
  model.set_weights(initial_weights)

warmup(model, x_train, y_train, x_test, y_test)
%time train_model(model, x_train, y_train, x_test, y_test)

scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Train on 50000 samples, validate on 10000 samples
50000/50000 [==============================] - 14s 281us/sample - loss: 2.0458 - accuracy: 0.2542 - val_loss: 1.7722 - val_accuracy: 0.3806
Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 9s 185us/sample - loss: 2.1345 - accuracy: 0.2117 - val_loss: 1.8806 - val_accuracy: 0.3368
Epoch 2/25
50000/50000 [==============================] - 9s 185us/sample - loss: 1.7852 - accuracy: 0.3593 - val_loss: 1.6491 - val_accuracy: 0.4163
Epoch 3/25
50000/50000 [==============================] - 9s 186us/sample - loss: 1.6493 - accuracy: 0.4046 - val_loss: 1.5709 - val_accuracy: 0.4416
Epoch 4/25
50000/50000 [==============================] - 9s 187us/sample - loss: 1.5651 - accuracy: 0.4352 - val_loss: 1.4745 - val_accuracy: 0.4754
Epoch 5/25
50000/50000 [==============================] - 9s 186us/sample - loss: 1.5023 - accuracy: 0.4590 - val_loss: 1.4032 - val_accuracy: 0.5021
Epoch 6/25

KeyboardInterrupt: 

10000/10000 [==============================] - 1s 147us/sample - loss: 1.2875 - accuracy: 0.5418
Test loss: 1.287472003173828
Test accuracy: 0.5418
